In [ ]:
import os
from dotenv import dotenv_values
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

env_vars = dotenv_values('.env')
os.environ['OPENAI_API_KEY'] = env_vars.get('OPENAI_API_KEY')

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100
)

loader = UnstructuredFileLoader('./files/o.txt')

len(loader.load_and_split(text_splitter=splitter))

